<a href="https://colab.research.google.com/github/nicolenair/capstone/blob/master/Capstone_LL_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install nltk
# nltk.download('averaged_perceptron_tagger')
# nltk.download('punkt')
import requests
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
books = ["gables", "avonlea", "island", "dreams"]
urls = ["https://www.gutenberg.org/files/45/45-0.txt", "https://www.gutenberg.org/files/47/47-0.txt", "https://www.gutenberg.org/files/51/51-0.txt", "http://www.gutenberg.org/cache/epub/544/pg544.txt"]

for i in range(len(books)):
    page = requests.get(urls[i])
    file = open("{}.txt".format(books[i]) , "w+")
    file.write(page.text)
    file.close()

file = open("gables.txt", "r")
gables = file.read()
file = open("avonlea.txt", "r")
avonlea = file.read()
file = open("island.txt", "r")
island = file.read()
file = open("dreams.txt", "r")
dreams = file.read()

In [0]:
#returns list of paragraphs in a book that can be innputted into train_classifier
def paragraph_list(book):
    current_total = 0
    chapter_lengths = []
    full_para_list = []
    lengths = [38, 0, 40, 0]
    if book==gables:
        n_chap = lengths[0]
    elif book == dreams:
        n_chap = lengths[2]
    for i in range(1, n_chap):
        if book == dreams:
            index_c1 = book[re.search("CHAPTER {}".format(i), book).span()[0] + 11: re.search("CHAPTER {}".format(i + 1), book).span()[0]]
        if book == gables:
            nums = ["I", "II","III","IV","V", "VI", "VII", "VIII", "IX", "X", "XI", "XII", "XIII","XIV", "XV", "XVI", "XVII", "XVIII", "XIX", "XX", "XXI", "XXII", "XXIII", "XXIV", "XXV", "XXVI","XXVII" ,"XXVIII","XXIX", "XXX", "XXXI", "XXXII", "XXXIII","XXXIV", "XXXV" ,"XXXVI", "XXXVII", "XXXVIII"]
            first1 = re.search("CHAPTER {}\W".format(nums[i-1]), gables).span()[0]
            first2 = re.search("CHAPTER {}\W".format(nums[i]), gables).span()[0]
            index_c1 = gables[re.search("CHAPTER {}\W".format(nums[i-1]), gables[first1 + 1:]).span()[0] + first1: re.search("CHAPTER {}\W".format(nums[i]), gables[first2 + 1:]).span()[0] + first2]
        index_c1 = re.sub("(\n)(\w)", " \g<2>", index_c1)
        
        chapter_lengths.append(len(index_c1) - current_total)
        split_para_c1 = [s.strip() for s in index_c1.splitlines() if len(s)>10]
        full_para_list.append(split_para_c1)
        current_total += len(index_c1)  
        
    return full_para_list

full_para_list = paragraph_list(dreams)

#data set, be very careful with cell!!!
target_dreams = [[0, 1, 0, 0, 6, 1, 0, 6, 0, 1, 6, 1, 6, 1, 6, 1, 6, 1, 6, 1, 6, 1, 6, 1, 6, 1, 6, 6, 1, 6, 0, 1, 6, 1], 
          [0, 0, 3, 2, 0, 2, 2, 3, 3, 2, 3, 0, 0, 7, 1, 7, 8, 0, 4, 1, 4 ,4, 1, 4, 4, 1, 4, 1, 4, 1, 4, 1, 4, 1], 
          [0, 2, 1, 3, 1, 3, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 0, 1, 2, 1, 2, 1, 2, 1, 0, 2, 1, 2, 2, 3, 2, 0,  0, 1, 9, 0, 1, 9, 1, 0, 10, 1, 10, 1, 10,0, 11, 1, 11, 1,11, 1,11,  1, 11, 1, 11, 12,1,0, 1, 1,0, 4, 1, 4, 1, 4, 1, 4, 1, 0],
          [0, 0, 1, 0, 0, 6, 3, 0, 2, 3, 6, 2, 6, 2, 1, 1, 2, 0, 0, 0], 
          [0, 0, 0, 1, 4, 1, 4, 0, 1, 0, 1, 4, 1, 4, 1, 1, 4, 1, 4, 0, 0, 4 ],
          [0, 0, 13, 0, 1, 1, 1, 0, 1, 0, 4, 0, 5, 0, 5, 0, 0, 0, 0, 5, 5, 14, 5, 13, 5, 13, 5, 13, 5, 13, 5, 1, 5, 0, 5, 5, 5, 14, 5, 5, 0, 4, 0, 5, 14, 4, 5, 13, 5],
          [0, 1, 4, 5, 1, 5, 5, 0, 5, 5, 5, 5, 5, 4, 14, 5, 5, 14, 5, 5, 5, 5, 5, 14, 1, 5, 5, 5, 5, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 1, 14, 5, 5, 1, 5, 5, 1, 5, 0, 1, 5, 0, 5, 1, 5, 1, 1, 5, 1, 5, 1, 1, 5, 5, 0, 5, 5, 13, 5, 0, 1],
          [0, 0, 1, 4, 1, 0, 1, 4, 0, 1, 4, 1, 4, 4, 0, 1, 4, 1, 4, 1, 0, 0, 15, 0, 15, 0, 15, 1, 15, 1, 15, 1, 15, 1, 15, 1, 15, 1, 15, 1, 15, 1, 15, 1, 15, 1, 0, 15, 0, 1, 15, 1, 15, 1, 15, 15, 15, 1, 0, 15, 1, 15, 1, 15, 1, 15, 1, 15, 1, 15, 15, 0, 15, 1, 15, 1, 15, 1, 15, 0, 1, 15, 1, 15], 
          [0, 0, 5, 0, 5, 5, 0, 5, 0, 1, 4, 1, 0, 1, 0, 4, 1, 4, 1, 0, 0, 5, 0, 5, 0, 5, 0, 5, 1, 0, 5, 4, 0, 5, 1, 5, 1, 5, 0, 5, 4, 5, 1, 5, 4, 0, 5, 0, 0, 0, 0, 0, 5, 5, 1, 1, 1, 0, 5, 0, 4, 1, 4, 1, 0], 
          [0, 1, 0, 1, 1, 0, 0, 1, 16, 0, 1, 16, 0, 1, 16, 1, 1, 16, 1, 16, 0, 16, 1, 16, 1, 16, 1, 16, 1, 16, 1, 16, 1, 16, 0, 1, 16, 1, 16, 1, 16, 0, 16, 1, 16, 1, 16, 1, 16, 0, 1, 16, 1, 0, 16, 0, 1, 16, 1, 16, 0, 4, 1, 4, 1, 4, 4, 1]]
    
target_gables = [[0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 0, 0, 2, 0, 3, 0, 2, 3, 0, 2, 0, 3, 3, 0, 2, 0, 3, 2, 3, 0, 2, 0],
                 [0, 0, 0, 0, 0, 0, 0, 0, 17, 0, 17, 18, 17, 0, 0, 0, 0, 1, 0, 18, 1, 0, 0, 1, 18, 1, 18, 1, 18, 1, 0, 18, 1, 18, 1, 18, 1, 0, 18, 0, 1, 18, 1, 18, 1, 18, 1, 0, 0, 0, 18, 0, 1, 18, 1, 18, 1, 0, 18, 1, 0, 18, 1, 18, 1, 18, 1, 18, 1, 0, 18, 1, 0, 1, 0, 18, 1, 0,0, 1, 0], 
                 [0, 0, 3, 18, 0, 3, 18, 3, 0, 1, 0, 3, 1, 0, 3, 0, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 18, 18, 3, 1, 3, 0, 3, 1, 3, 1, 3, 1, 18, 0, 0, 3, 0, 1, 3, 0, 0, 3, 0, 1, 0, 0, 3, 18, 3, 18, 3, 0, 18, 3, 18, 3, 18, 18, 3, 18, 3, 18, 0],
                 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 3, 1, 3, 3, 0, 1, 3, 0, 0, 0, 0, 3, 1, 3, 1, 3, 3, 0, 0, 3, 1, 3, 1, 3, 1, 3, 0, 3, 0, 3, 0, 0, 3, 0], 
                 [0, 1, 3, 0, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 1, 1, 0, 3, 1, 3, 0, 3, 0, 1, 3, 1, 1], 
                 [0, 0, 19, 1, 3, 1, 19, 19, 17, 19, 3, 19, 0, 0, 19, 19, 0, 0, 19, 19, 0, 17, 17, 0, 3, 17, 0, 1, 3, 1, 0, 3, 1, 0, 0, 18, 3, 0, 18, 3, 18, 0, 3],
                 [0, 0, 3, 1, 3, 1, 0, 3, 1, 0, 3, 1, 3, 1, 0, 3, 1, 3, 0, 1, 0, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 3, 1, 0, 18]]
name_mapping = {"narrator":0, "Anne":1, "Rachel":2, "Marilla":3, "Gilbert":4, "Captain Jim":5, "Diana":6, "Mrs. Harmon Andrews":7, "Mrs. Jasper":8, "Paul":9, "Charlotta the Fourth":10, "Philippa":11, "Miss Patty":12, "Mrs. Doctor":13, "Doctor Dave":14, "Cornelia":15, "Leslie":16, "unnamed char": 17, "Matthew":18, "Mrs. Spencer":19}

def remove_names(full_para_list, target):
    for o, i in enumerate(full_para_list):
        for t, e in enumerate(i):
            for k in name_mapping.keys():
                if target[o][t]==name_mapping[k]:
                    full_para_list[o][t] = re.sub(k, "", full_para_list[o][t])
                    full_para_list[o][t] = re.sub(k.lower(), "", full_para_list[o][t])
    return full_para_list
#have a more comprehensive way names could appear
#also try to do the said thing, rather than just removing all names

full_para_list = remove_names(full_para_list[:10], target_dreams)

In [0]:
full_para_list_gables = paragraph_list(gables)
full_para_list_dreams = paragraph_list(dreams)

In [25]:
i = 57
print(full_para_list_dreams[9][i])
print(target_dreams[9][i])

"Oh, I do--I do," exclaimed Leslie, with an eagerness which seemed to burst forth and beat down some restraint that had been imposed on it.
16


In [26]:
full_para_list_dreams[9][67]

'"Don\'t let us judge her till we know," pleaded Anne.  "I don\'t believe her case is so ordinary.  You will understand her fascination when you meet her, Gilbert.  It is a thing quite apart from her beauty.  I feel that she possesses a rich nature, into which a friend might enter as into a kingdom; but for some reason she bars every one out and shuts all her possibilities up in herself, so that they cannot develop and blossom.  There, I\'ve been struggling to define her to myself ever since I left her, and that is the nearest I can get to it.  I\'m going to ask Miss Cornelia about her."'

In [0]:
import nltk

word_list_dreams = dreams.split()
# word_list_dreams[:10]
postagged_word_list_dreams = nltk.pos_tag(word_list_dreams)

list_of_tags = [i[1] for i in postagged_word_list_dreams]
x_tagname = []
y_tagcount = []
for i in list(set(list_of_tags)):
  y_tagcount.append(list_of_tags.count(i))
  x_tagname.append(i)
  
full_dreams_tags = pd.DataFrame({"tagname":x_tagname, "tagcount":y_tagcount})

In [28]:
words_anne_dreams = []
for i in range(len(target_dreams)):
  for e in range(len(target_dreams[i])):
#      print(i, e)
     if target_dreams[i][e]==1:
        words_anne_dreams.extend(full_para_list_dreams[i][e].split())
        
tag_anne_dreams = nltk.pos_tag(words_anne_dreams)

list_of_tags_anne_dreams = [i[1] for i in tag_anne_dreams]
x_tagname = []
y_tagcount = []
for i in list(set(list_of_tags_anne_dreams)):
  y_tagcount.append(list_of_tags_anne_dreams.count(i))
  x_tagname.append(i)
  
# full_gables_tags.loc["anne_tagname"] = x_tagname
# full_gables_tags.loc["anne_tagcount"] = y_tagcount
full_dreams_tags["anne_tagcount"] = 0
for i in range(len(full_dreams_tags)):
  if full_dreams_tags["tagname"][i] in x_tagname:
    full_dreams_tags["anne_tagcount"][i] = y_tagcount[x_tagname.index(full_dreams_tags["tagname"][i])]
    
    
full_dreams_tags["anne_tagcount"] = full_dreams_tags["anne_tagcount"]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [0]:
import nltk

word_list_gables = gables.split()
# word_list_dreams[:10]
postagged_word_list_gables = nltk.pos_tag(word_list_gables)

list_of_tags = [i[1] for i in postagged_word_list_gables]
x_tagname = []
y_tagcount = []
for i in list(set(list_of_tags)):
  y_tagcount.append(list_of_tags.count(i))
  x_tagname.append(i)
  
full_gables_tags = pd.DataFrame({"tagname":x_tagname, "tagcount":y_tagcount})

In [30]:
words_anne_gables = []
for i in range(len(target_gables)):
  for e in range(len(target_gables[i])):
     # print(i, e)
     if target_gables[i][e]==1:
        words_anne_gables.extend(full_para_list_gables[i][e].split())
        
tag_anne_gables = nltk.pos_tag(words_anne_gables)

list_of_tags_anne = [i[1] for i in tag_anne_gables]
x_tagname = []
y_tagcount = []
for i in list(set(list_of_tags_anne)):
  y_tagcount.append(list_of_tags_anne.count(i))
  x_tagname.append(i)
  
# full_gables_tags.loc["anne_tagname"] = x_tagname
# full_gables_tags.loc["anne_tagcount"] = y_tagcount
full_gables_tags["anne_tagcount"] = 0
for i in range(len(full_gables_tags)):
  if full_gables_tags["tagname"][i] in x_tagname:
    full_gables_tags["anne_tagcount"][i] = y_tagcount[x_tagname.index(full_gables_tags["tagname"][i])]
    
    
full_gables_tags["anne_tagcount"] = full_gables_tags["anne_tagcount"]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [49]:
for i in full_gables_tags["tagname"]:
  if i not in full_dreams_tags["tagname"]:
    full_dreams_tags["tagname"][i] = 0.01

#   if full_gables_tags["tagcount"]
    
for i in full_dreams_tags["tagname"]:
  if i not in full_gables_tags["tagname"]:
    full_gables_tags["tagname"][i] = 0.01
    
full_dreams_tags.loc[full_dreams_tags["tagcount"]==0, "tagcount"] = 0.0001
full_gables_tags.loc[full_gables_tags["tagcount"]==0, "tagcount"] = 0.0001
full_dreams_tags.loc[full_dreams_tags["anne_tagcount"]==0, "anne_tagcount"] = 0.0001
full_gables_tags.loc[full_gables_tags["anne_tagcount"]==0, "anne_tagcount"] = 0.0001


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
# full_dreams_tags

In [50]:
from scipy.stats import chi2_contingency
print(chi2_contingency(full_gables_tags.iloc[:,1:])[:2])
print(chi2_contingency(full_dreams_tags.iloc[:,1:])[:2])
print(chi2_contingency(pd.DataFrame({"0":np.array(full_gables_tags.iloc[:,2]), "1": np.array(full_dreams_tags.iloc[:,2])}))[:2])
print(chi2_contingency(pd.DataFrame({"0":np.array(full_gables_tags.iloc[:,1]), "1": np.array(full_dreams_tags.iloc[:,1])}))[:2])
print(chi2_contingency(pd.DataFrame({"0":np.array(full_gables_tags.iloc[:,1]), "1": np.array(full_dreams_tags.iloc[:,2])}))[:2])
print(chi2_contingency(pd.DataFrame({"0":np.array(full_gables_tags.iloc[:,2]), "1": np.array(full_dreams_tags.iloc[:,1])}))[:2])

(92.63375045613837, 6.953700226936897e-07)
(120.58595134123259, 4.663514636757873e-11)
(4239.682410937771, 0.0)
(85244.524111842, 0.0)
(25323.687248796046, 0.0)
(34618.407398429285, 0.0)


In [51]:
full_dreams_tags

,tagname,tagcount,anne_tagcount
0,SYM,1.0,0.0001
1,CD,529.0,27.0000
2,RBR,55.0,3.0000
3,VBN,1730.0,97.0000
4,VBZ,1583.0,112.0000
5,POS,51.0,6.0000
6,PDT,136.0,6.0000
7,PRP$,1711.0,89.0000
8,JJ,6741.0,394.0000
9,WRB,561.0,34.0000


In [0]:
pd.DataFrame({"0":np.array(full_gables_tags.iloc[:,2]), "1": np.array(full_dreams_tags.iloc[:,2])})

,0,1
0,1,0
1,1,0
2,0,237
3,261,469
4,491,123
5,105,118
6,99,83
7,65,388
8,7,6
9,414,1


0       0
1       0
2     237
3     469
4     123
5     118
6      83
7     388
8       6
9       1
10    674
11    134
12     34
13    545
14      3
15     89
16     12
17     23
18     97
19    262
20     28
21     69
22      3
23    112
24    311
25      6
26     16
27    254
28      7
29      0
30    197
31      6
32      0
33      0
34      5
35    394
36     27
Name: anne_tagcount, dtype: int64